In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table):
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursorPROC.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            connPROC.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connPROC.commit()
    print("PUSH DATA: DONE")

dataset_path = 'F:\Work\Caohoc_2024_2026\PTPM_project\imdb_dataset'

In [3]:
'''
-- 23032025 - hieudd - Đẩy dữ liệu thẳng vào DB, Dữ liệu clear
'''
# title_ratings = pd.read_csv(os.path.join(dataset_path,'title.ratings.tsv'),sep="\t")
# title_ratings

'\n-- 23032025 - hieudd - Đẩy dữ liệu thẳng vào DB, Dữ liệu clear\n'

In [4]:
# title_crew = pd.read_csv(os.path.join(dataset_path,'title.crew.tsv'),sep="\t")
# title_crew['directors'] = title_crew['directors'].replace(r'\N', np.nan)
# title_crew['writers'] = title_crew['writers'].replace(r'\N', np.nan)
# title_crew

In [ ]:
'''
-- 23032025 - hieudd - Xóa 9 dòng chứa primaryName bị null
-- 23032025 - hieudd - Tách data thành 3 bảng: info, profession, fntitle
'''
name_raw = pd.read_csv(os.path.join(dataset_path,'name.basics.tsv'),sep="\t")
name_raw['deathYear'] = name_raw['deathYear'].replace(r'\N', np.nan)
name_raw = name_raw[name_raw['primaryName'].isnull()==False]
name_raw.reset_index(drop=True,inplace=True)

name_info = name_raw[['nconst','primaryName','birthYear','deathYear']]                  # Phân vùng bảng name_info đẩy thằng vào DB - Dữ liệu clean
name_info

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"
...,...,...,...,...,...,...
14262552,nm9993714,Romeo del Rosario,\N,NaN,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
14262553,nm9993716,Essias Loberg,\N,NaN,\N,\N
14262554,nm9993717,Harikrishnan Rajan,\N,NaN,cinematographer,tt8736744
14262555,nm9993718,Aayush Nair,\N,NaN,cinematographer,tt8736744


,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,NaN
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007
...,...,...,...,...
14262552,nm9993714,Romeo del Rosario,\N,NaN
14262553,nm9993716,Essias Loberg,\N,NaN
14262554,nm9993717,Harikrishnan Rajan,\N,NaN
14262555,nm9993718,Aayush Nair,\N,NaN


In [ ]:
# name_profession = name_raw[['nconst','primaryProfession']]
name_prof_dask = dd.from_pandas(name_raw[['nconst','primaryProfession']], npartitions=20)                       # Tạo dask dataframe từ dataframe pandas với 20 phân vùng bộ nhớ đệm (npartitions)
df_expanded = name_prof_dask['primaryProfession'].apply(lambda x: x.split(','), meta=('x', 'object'))           # sử dụng apply() tách theo dấu , với dask dataframe - không giữ danh sách trong RAM
df_expanded = df_expanded.apply(pd.Series, meta={'profession1':str,'profession2':str,'profession3':str})        # chuyển 
df_expanded
# type(name_prof_dask)
# name_prof_dask

,profession1,profession2,profession3
npartitions=20,,,
0,object,object,object
713128,...,...,...
...,...,...,...
13549430,...,...,...
14262556,...,...,...


In [ ]:
df_expanded = name_prof_dask['primaryProfession'].apply(lambda x: x.split(','), meta=('x', 'object'))
df_expanded = df_expanded.apply(pd.Series, meta={0: str, 1: str, 2: str})
df_expanded

Dask Series Structure:
npartitions=20
0           object
713128         ...
             ...  
13549430       ...
14262556       ...
Dask Name: apply, 3 expressions
Expr=Apply(frame=df['primaryProfession'], function=<function <lambda> at 0x0000017D4D788E50>, meta=('x', 'object'))

In [ ]:
name_profession_1P = name_prof_dask['primaryProfession'].apply(lambda x: x.split(','), meta=('x', 'object'))
name_profession_1P

TypeError: <lambda>() got an unexpected keyword argument 'meta'

In [3]:
title_basics = pd.read_csv(os.path.join(dataset_path,'title.basics.tsv'),sep="\t")
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
...,...,...,...,...,...,...,...,...,...
11526090,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,\N,\N,Drama
11526091,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,Drama
11526092,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,Drama
11526093,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
set(title_basics['titleType'])

{'movie',
 'short',
 'tvEpisode',
 'tvMiniSeries',
 'tvMovie',
 'tvPilot',
 'tvSeries',
 'tvShort',
 'tvSpecial',
 'video',
 'videoGame'}

In [ ]:
title_ratings = pd.read_csv(os.path.join(dataset_path,'title.ratings.tsv'),sep="\t")
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,2142
1,tt0000002,5.5,290
2,tt0000003,6.4,2179
3,tt0000004,5.3,186
4,tt0000005,6.2,2912
...,...,...,...
1546480,tt9916730,7.0,12
1546481,tt9916766,7.2,24
1546482,tt9916778,7.2,37
1546483,tt9916840,6.8,12


In [12]:
title_ratings.columns.to_list()

['tconst', 'averageRating', 'numVotes']

In [13]:
cols = "`,`".join([str(i) for i in title_ratings.columns.tolist()])
cols

'tconst`,`averageRating`,`numVotes'

In [ ]:
SQL_push('public.rating',title_ratings)

In [8]:
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546485 entries, 0 to 1546484
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1546485 non-null  object 
 1   averageRating  1546485 non-null  float64
 2   numVotes       1546485 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 35.4+ MB


In [9]:
len(set(title_ratings['tconst']))

1546485

In [7]:
title_episode = pd.read_csv(os.path.join(dataset_path,'title.episode.tsv'),sep="\t")
title_episode

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,\N,\N
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,\N,\N
4,tt0043426,tt0040051,3,42
...,...,...,...,...
8867244,tt9916846,tt1289683,3,18
8867245,tt9916848,tt1289683,3,17
8867246,tt9916850,tt1289683,3,19
8867247,tt9916852,tt1289683,3,20


In [ ]:
title_akas = pd.read_csv(os.path.join(dataset_path,'title.akas.tsv'),sep="\t")
title_akas

In [2]:
title_principals = pd.read_csv(os.path.join(dataset_path,'title.principals.tsv'),sep="\t")
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N
...,...,...,...,...,...,...
91510827,tt9916880,17,nm0996406,director,principal director,\N
91510828,tt9916880,18,nm1482639,writer,\N,\N
91510829,tt9916880,19,nm2586970,writer,books,\N
91510830,tt9916880,20,nm1594058,producer,producer,\N
